In [ ]:
import pandas as pd
import s3fs
from smart_open import open
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import metrics
import torch
from transformers import *
import numpy as np
import ast
import time

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
user_tokens = pd.read_csv('s3://recsyschallenge2020/user_tokens.csv')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
model = BertModel.from_pretrained('/dev/bert/')

In [ ]:
def calculate_average(row):
    sum_tensors = torch.zeros([768], dtype=torch.float32)
    tweet_token_list = ast.literal_eval(row)
    for token_list in tweet_token_list:
        token_list_embeddings = model(torch.tensor(list(map(int, token_list.split('\t')))).unsqueeze(0))[0][0][0]
        sum_tensors = sum_tensors + token_list_embeddings
    avg = sum_tensors/len(row)
    return avg

In [ ]:
user_tokens['avg_embeddings'] = user_tokens['text_ tokens'].apply(lambda x: calculate_average(x))

In [ ]:
user_tokens.to_csv('s3://recsyschallenge2020/user_tokens_embeddins.csv', index = False)

In [ ]:
user_tokens.head()

In [ ]:
len(user_tokens)

In [ ]:
input_ids = torch.tensor([101, 108, 31278, 90939, 70325, 196, 199, 71436, 10107, 29190, 10107, 106, 16680, 68314, 10153, 17015, 15934, 10104, 108, 10233, 12396, 14945, 10107, 10858, 11405, 13600, 13597, 169, 57343, 64482, 119, 119, 119, 100, 11741, 16381, 10109, 68830, 10110, 20886, 108, 10233, 11127, 21768, 100, 14120, 131, 120, 120, 188, 119, 11170, 120, 12132, 10884, 10157, 11490, 12022, 10113, 10731, 10729, 11565, 14120, 131, 120, 120, 188, 119, 11170, 120, 162, 11211, 11703, 12022, 11211, 10240, 44466, 100886, 102]).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0][0][0]  # The last hidden-state is the first element of the output tuple

In [ ]:
model(torch.tensor(list(map(int, ast.literal_eval(user_tokens.loc[0]['text_ tokens']).split('\t')))).unsqueeze(0))[0][0][0]

In [ ]:
it = 0

start = time.time()

# your code

# end
sum_tensors = torch.zeros([768], dtype=torch.float32)
for index, row in user_tokens.iterrows():
    #print(len(row))
    tweet_token_list = ast.literal_eval(row['text_ tokens'])
    for token_list in tweet_token_list:
        token_list_embeddings = model(torch.tensor(list(map(int, token_list.split('\t')))).unsqueeze(0))[0][0][0]
        sum_tensors = sum_tensors + token_list_embeddings
    if it %100 == 0:
        print(f'Time: {time.time() - start}')
        print(it)
    it = it + 1
    print(it)
    avg = sum_tensors/len(row)

In [ ]:
sum_tensors = torch.zeros([768], dtype=torch.float32)

In [ ]:
sum_tensors

In [ ]:
import dask.dataframe as ddf

In [ ]:
df_dask = ddf.from_pandas(user_tokens, npartitions=32)   # where the number of partitions is the number of cores you want to use

In [ ]:
df_dask.apply(lambda x: calculate_average(x), axis  = 1).compute(scheduler='multiprocessing')